***GENERATED CODE FOR test2 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'job', 'transformation_label': 'String Indexer'}], 'feature': 'job', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '17', 'mean': '', 'stddev': '', 'min': 'admin.', 'max': 'technician', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'job'}, {'feature_label': 'job', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('job')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'marital', 'transformation_label': 'String Indexer'}], 'feature': 'marital', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17', 'mean': '', 'stddev': '', 'min': 'married', 'max': 'single', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'marital'}, {'feature_label': 'marital', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('marital')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'education', 'transformation_label': 'String Indexer'}], 'feature': 'education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17', 'mean': '', 'stddev': '', 'min': 'primary', 'max': 'tertiary', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'education'}, {'feature_label': 'education', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('education')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'default', 'transformation_label': 'String Indexer'}], 'feature': 'default', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'no', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'default'}, {'feature_label': 'default', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('default')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'housing', 'transformation_label': 'String Indexer'}], 'feature': 'housing', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'housing'}, {'feature_label': 'housing', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('housing')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'loan', 'transformation_label': 'String Indexer'}], 'feature': 'loan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'loan'}, {'feature_label': 'loan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('loan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'contact', 'transformation_label': 'String Indexer'}], 'feature': 'contact', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17', 'mean': '', 'stddev': '', 'min': 'cellular', 'max': 'unknown', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'contact'}, {'feature_label': 'contact', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('contact')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'month', 'transformation_label': 'String Indexer'}], 'feature': 'month', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17', 'mean': '', 'stddev': '', 'min': 'aug', 'max': 'may', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'month'}, {'feature_label': 'month', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('month')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'poutcome', 'transformation_label': 'String Indexer'}], 'feature': 'poutcome', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17', 'mean': '', 'stddev': '', 'min': 'failure', 'max': 'unknown', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'poutcome'}, {'feature_label': 'poutcome', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('poutcome')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'deposit', 'transformation_label': 'String Indexer'}], 'feature': 'deposit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'deposit'}, {'feature_label': 'deposit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('deposit')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run test2Hooks.ipynb
try:
	#sourcePreExecutionHook()

	bankdeposittest = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/BankDepositTest.csv', 'filename': 'BankDepositTest.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://10.0.5.54', 'port': '50070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(bankdeposittest)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run test2Hooks.ipynb
try:
	#transformationPreExecutionHook()

	fee = TransformationMain.run(bankdeposittest,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "42.29", "stddev": "10.1", "min": "29", "max": "59", "missing": "0"}, "updatedLabel": "age"}, {"transformationsData": [{"feature_label": "job", "transformation_label": "String Indexer"}], "feature": "job", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "admin.", "max": "technician", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "job"}, {"transformationsData": [{"feature_label": "marital", "transformation_label": "String Indexer"}], "feature": "marital", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "married", "max": "single", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "marital"}, {"transformationsData": [{"feature_label": "education", "transformation_label": "String Indexer"}], "feature": "education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "primary", "max": "tertiary", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "education"}, {"transformationsData": [{"feature_label": "default", "transformation_label": "String Indexer"}], "feature": "default", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "no", "max": "no", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "default"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "balance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "689.59", "stddev": "900.5", "min": "-56", "max": "2476", "missing": "0"}, "updatedLabel": "balance"}, {"transformationsData": [{"feature_label": "housing", "transformation_label": "String Indexer"}], "feature": "housing", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "housing"}, {"transformationsData": [{"feature_label": "loan", "transformation_label": "String Indexer"}], "feature": "loan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "loan"}, {"transformationsData": [{"feature_label": "contact", "transformation_label": "String Indexer"}], "feature": "contact", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "cellular", "max": "unknown", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "contact"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "10.53", "stddev": "6.97", "min": "5", "max": "28", "missing": "0"}, "updatedLabel": "day"}, {"transformationsData": [{"feature_label": "month", "transformation_label": "String Indexer"}], "feature": "month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "aug", "max": "may", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "duration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "741.0", "stddev": "568.63", "min": "25", "max": "1689", "missing": "0"}, "updatedLabel": "duration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "2.06", "stddev": "1.52", "min": "1", "max": "7", "missing": "0"}, "updatedLabel": "campaign"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pdays", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "41.35", "stddev": "119.56", "min": "-1", "max": "361", "missing": "0"}, "updatedLabel": "pdays"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17", "mean": "0.24", "stddev": "0.75", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "previous"}, {"transformationsData": [{"feature_label": "poutcome", "transformation_label": "String Indexer"}], "feature": "poutcome", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "failure", "max": "unknown", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "poutcome"}, {"transformationsData": [{"feature_label": "deposit", "transformation_label": "String Indexer"}], "feature": "deposit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "deposit"}]}))

	#transformationPostExecutionHook(fee)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run test2Hooks.ipynb
try:
	#sinkPreExecutionHook()

	muhammadoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/643fc46826cb4b9578b84bee/part-00000-b2eb4ded-bcbd-4880-a024-6cc3ffbf77fd-c000.csv', 'samplecount': 17, 'originalcount': 17, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/BankDepositTest.csv', 'pathOnly': '/Muhammad', 'project_id': '643fbc3d26cb4b9578b847f4', 'parent_id': '643fbc3d26cb4b9578b847f4', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cb7', 'field': 'age', 'alias': 'age', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cb8', 'field': 'balance', 'alias': 'balance', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cb9', 'field': 'day', 'alias': 'day', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cba', 'field': 'duration', 'alias': 'duration', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cbb', 'field': 'campaign', 'alias': 'campaign', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cbc', 'field': 'pdays', 'alias': 'pdays', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cbd', 'field': 'previous', 'alias': 'previous', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cbe', 'field': 'job_stringindexer', 'alias': 'job_stringindexer', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cbf', 'field': 'marital_stringindexer', 'alias': 'marital_stringindexer', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cc0', 'field': 'education_stringindexer', 'alias': 'education_stringindexer', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cc1', 'field': 'default_stringindexer', 'alias': 'default_stringindexer', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cc2', 'field': 'housing_stringindexer', 'alias': 'housing_stringindexer', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cc3', 'field': 'loan_stringindexer', 'alias': 'loan_stringindexer', 'type': 'numeric', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cc4', 'field': 'contact_stringindexer', 'alias': 'contact_stringindexer', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cc5', 'field': 'month_stringindexer', 'alias': 'month_stringindexer', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cc6', 'field': 'poutcome_stringindexer', 'alias': 'poutcome_stringindexer', 'type': 'numeric', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cc7', 'field': 'deposit_stringindexer', 'alias': 'deposit_stringindexer', 'type': 'numeric', 'position': '16'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84ca6', 'field': 'age', 'alias': 'age', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84ca7', 'field': 'job', 'alias': 'job', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84ca8', 'field': 'marital', 'alias': 'marital', 'type': 'String', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84ca9', 'field': 'education', 'alias': 'education', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84caa', 'field': 'default', 'alias': 'default', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cab', 'field': 'balance', 'alias': 'balance', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cac', 'field': 'housing', 'alias': 'housing', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cad', 'field': 'loan', 'alias': 'loan', 'type': 'String', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cae', 'field': 'contact', 'alias': 'contact', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84caf', 'field': 'day', 'alias': 'day', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cb0', 'field': 'month', 'alias': 'month', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cb1', 'field': 'duration', 'alias': 'duration', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cb2', 'field': 'campaign', 'alias': 'campaign', 'type': 'numeric', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cb3', 'field': 'pdays', 'alias': 'pdays', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cb4', 'field': 'previous', 'alias': 'previous', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cb5', 'field': 'poutcome', 'alias': 'poutcome', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '643fc73726cb4b9578b84cb6', 'field': 'deposit', 'alias': 'deposit', 'type': 'String', 'position': '16'}], 'server': 'https://dev.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'testfile.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-079f23fe-2abd-4c64-8a3a-963b25b60d07-c000.csv', 'results_url': 'http://dev.numtra.com:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
